In [236]:
import openpyxl
import pandas as pd
import os
import re

total_wi = pd.read_excel("Work_plan_3gpp_230110.xlsm")
# wb = openpyxl.load_workbook('Work_plan_3gpp_230110.xlsm')
# ws = wb['Work_plan_3gpp']
# result = pd.read_excel("result_with_id.xlsx")
# date: 2023/2/19

## 1. 探索Work_plan_3gpp_230110.xlsm檔案

In [237]:
print('Work plan表格中所有欄位: ')
print([i for i in total_wi.columns])

'''

3GPP shall consist of a Project Co-ordination Group (PCG) and Technical Specification Groups (TSGs). 
The Technical Specification Groups may establish Working Groups if required.

Technical Specification Groups (TSGs) 
Working Groups (WG)
TSG Sub Working Groups (SWG)


The 3GPP Work Programme shall consist of Work Items defined by the TSGs.

'''

Work plan表格中所有欄位: 
['ID', 'Unique_ID', 'Name', 'Acronym', 'OutlineLevel', 'Release', 'ResourceNames', 'Start', 'Finish', '%Complete', 'Hyperlink', 'Status Report', 'WI rapporteur name', 'WI rapporteur e-mail', 'Notes', 'Impacted TSs and TRs', 'TSG approved', 'PCG approved', 'TSG stopped', 'PCG stopped', 'Created', 'last modif']


'\n\n3GPP shall consist of a Project Co-ordination Group (PCG) and Technical Specification Groups (TSGs). \nThe Technical Specification Groups may establish Working Groups if required.\n\nTechnical Specification Groups (TSGs) \nWorking Groups (WG)\nTSG Sub Working Groups (SWG)\n\n\nThe 3GPP Work Programme shall consist of Work Items defined by the TSGs.\n\n'

In [239]:
content = {}
valid_lines = 0

wi_with_links = {}
wi_id_without_links = []

for i in range(1, len(ws['K'])+1):
    if ws[f'B{i}'].value != 0 and ws[f'B{i}'].value != 'Unique_ID': 
        valid_lines += 1
        
        if ws.cell(i, 11).hyperlink:
            wi_with_links[ws[f'B{i}'].value] = {
                        'Unique_ID': ws[f'B{i}'].value,
                        'release': ws[f'F{i}'].value,
                        'Hyperlink': ws.cell(i, 11).hyperlink.target,
                        'WI rapporteur name': ws[f'K{i}'].value,
                        'Supporting Individual Members':[] 
                    }
            
        elif not ws.cell(i, 11).hyperlink:
            wi_id_without_links.append(ws[f'B{i}'].value)


print(f'共{valid_lines}個 work items，其中{len(wi_with_links)}筆附有連結、{len(wi_id_without_links)}未附連結')
print('不重複的連結: ', len(set([value['Hyperlink'] for key, value in wi_with_links.items()])), ' => 全部下載')

共8143個 work items，其中5087筆附有連結、3056未附連結
不重複的連結:  2814  => 全部下載


## 2. 探索爬取下來的supporting members檔案

In [240]:
print('已下載不重複的work item數量: ', len(set(result['unique_ID'])))
print('有', len(wi_with_links) - len(set(result['unique_ID'])), '個work item下載不到資料')

# 刪除Work item名稱、檔案名稱
result = result.drop(['title', 'file'], axis=1)
# 刪除重複值
result = result.drop_duplicates()

result.head()

已下載不重複的work item數量:  2636
有 2451 個work item下載不到資料


,unique_ID,name,rapporteur,supporting member
0,860006,SP-200886,"gupta, nishant, samsung, nishant.gup@samsung.com",samsung
1,860006,SP-200886,"gupta, nishant, samsung, nishant.gup@samsung.com",airbus
2,860006,SP-200886,"gupta, nishant, samsung, nishant.gup@samsung.com",alibaba
3,860006,SP-200886,"gupta, nishant, samsung, nishant.gup@samsung.com",at&t
4,860006,SP-200886,"gupta, nishant, samsung, nishant.gup@samsung.com",bdbos


## 3.整理work item overview

In [241]:
total_wi = pd.read_excel("Work_plan_3gpp_230110.xlsm")

all_columns = [i for i in total_wi.columns]
need_columns = ['Unique_ID','Release','ResourceNames', 'Start', 'Finish', '%Complete','WI rapporteur name', 'WI rapporteur e-mail', 'Impacted TSs and TRs', 'Created', 'last modif']

wi_overview = total_wi.drop([i for i in all_columns if i not in need_columns], axis=1)[total_wi['Unique_ID'] != 0]
wi_overview.head()

,Unique_ID,Release,ResourceNames,Start,Finish,%Complete,WI rapporteur name,WI rapporteur e-mail,Impacted TSs and TRs,Created,last modif
5,970044,Rel-19,S1,2022-09-01,2022-09-19,1.0,"Yuying Zhang, China Telecom",zhangyy45@chinatelecom.cn,22.261,2022-09-09,2022-09-09
6,970043,Rel-19,S1,2022-09-01,2022-09-19,1.0,"Saha, Anindya, Saankhya Labs",anindya@saankhyalabs.com,22.261,2022-09-09,2022-09-09
7,970042,Rel-19,S1,2022-09-01,2022-09-19,1.0,"Singh, Ray, Peraton Labs",rsingh@peratonlabs.com,22.153,2022-09-09,2022-09-09
8,970041,Rel-19,S1,2022-09-01,2022-09-19,1.0,"Yinglin Chen, China Telecom",chenyl37@chinatelecom.cn,22.261,2022-09-09,2022-09-09
9,960020,Rel-19,S1,2022-06-02,2023-03-03,0.4,"LEE, Ki-Dong, LG Electronics",kidong.lee@lge.com,22.916,2022-06-02,2022-12-10


### 新增WIR所屬企業：從WIR name, WIR email整理

In [1]:
condition = wi_overview['WI rapporteur name'].str.contains(',')
selected_rows = wi_overview[condition == False]
wi_overview.drop(selected_rows.index, inplace=True)
selected_rows['WI rapporteur name'] = selected_rows['WI rapporteur name'].str.lower()

big_companys = ['siemens', 'ntt', 'sagem', 'softbank', 'telefónica', 'nokia', 'huawei', 'qualcomm', 'ericsson', 'samsung', 'motorola', 'china mobile', 'at&t','china telecom', 'cisco', 'intel', 'interdigital', 'lg', 'nec', 'oppo', 'orange', 'docomo', 'alcatel-lucent', 'airbus','vodafone', 'vivo','zte', 'apple','lenovo','trueposition', 'fujitsu', 't-mobile', 'trueposition', 'futurewei', 'telefonica', 'xiaomi', 'philips']

for company in big_companys:
        selected_rows.replace({'WI rapporteur name': fr'.*{company}.*$'}, {'WI rapporteur name': f'{company}'}, regex=True,inplace=True)

wi_overview = pd.concat([wi_overview, selected_rows])

In [244]:
rapporteur = []
big_companys = ['nokia', 'huawei', 'qualcomm', 'ericsson', 'samsung', 'motorola', 'china mobile', 'at&t','china telecom', 'cisco', 'intel', 'interdigital', 'lg electronics', 'nec', 'oppo', 'orange', 'docomo', 'alcatel-lucent', 'airbus','vodafone', 'vivo','zte', 'apple','lenovo','trueposition', 'fujitsu', 't-mobile', 'trueposition', 'futurewei', 'telefonica', 'xiaomi', 'philips']

def clean(i):
    if re.search(r'\Wat', i) and len(i)> 14:
        i = re.sub(r'\W+dot\W+', '.', i)
        return re.sub(r'at\W', '!', i).split('!')[1].strip().split('.')[0]
    else:
        return i.strip().lower()
    

for index, row in wi_overview.iterrows():
    
    if '@' in str(row['WI rapporteur name']) :
        rapporteur.append(clean(row['WI rapporteur name'].split('@')[1].split('.')[0]))


    elif ',' in str(row['WI rapporteur name']) or ';' in str(row['WI rapporteur name']):
        
        # email是nan
        if len(str(row['WI rapporteur e-mail'])) < 4:
            rapporteur.append(clean(row['WI rapporteur name']))
            
        # email不包含'@'
        elif '@' not in str(row['WI rapporteur e-mail']):
            rapporteur.append(clean(row['WI rapporteur e-mail']))    
            
        # 只有一個email
        elif len(str(row['WI rapporteur e-mail']).split('@')) == 2:
            rapporteur.append(clean(str(row['WI rapporteur e-mail']).split('@')[1].split('.')[0]))

        else:
            rapporteur.append(clean(', '.join([i.split('.')[0] for i in str(row['WI rapporteur e-mail']).split('@')[1:]])))

    else:
        
#         print(str(i).strip())
        rapporteur.append(clean(str(row['WI rapporteur name']).strip()))
        
print(len(wi_overview))
print(len(rapporteur))

8143
8143


In [245]:
wi_overview.insert(3, 'rapporteur', rapporteur)
# wi_overview = wi_overview.drop(['WI rapporteur name', 'WI rapporteur e-mail'], axis=1)
wi_overview

,Unique_ID,Release,ResourceNames,rapporteur,Start,Finish,%Complete,WI rapporteur name,WI rapporteur e-mail,Impacted TSs and TRs,Created,last modif
5,970044,Rel-19,S1,chinatelecom,2022-09-01,2022-09-19,1.0,"Yuying Zhang, China Telecom",zhangyy45@chinatelecom.cn,22.261,2022-09-09,2022-09-09
6,970043,Rel-19,S1,saankhyalabs,2022-09-01,2022-09-19,1.0,"Saha, Anindya, Saankhya Labs",anindya@saankhyalabs.com,22.261,2022-09-09,2022-09-09
7,970042,Rel-19,S1,peratonlabs,2022-09-01,2022-09-19,1.0,"Singh, Ray, Peraton Labs",rsingh@peratonlabs.com,22.153,2022-09-09,2022-09-09
8,970041,Rel-19,S1,chinatelecom,2022-09-01,2022-09-19,1.0,"Yinglin Chen, China Telecom",chenyl37@chinatelecom.cn,22.261,2022-09-09,2022-09-09
9,960020,Rel-19,S1,lge,2022-06-02,2023-03-03,0.4,"LEE, Ki-Dong, LG Electronics",kidong.lee@lge.com,22.916,2022-06-02,2022-12-10
...,...,...,...,...,...,...,...,...,...,...,...,...
8264,60158,R99,T2,-,1999-01-01,1999-12-31,1.0,-,-,34.907,2009-07-16,2009-07-06
8265,60159,R99,T2,-,1999-01-01,1999-12-31,1.0,-,-,34.925,2009-07-16,2009-07-06
8266,60160,R99,T2,-,1999-01-01,1999-12-31,1.0,-,-,"21.810, 21.910",2009-07-16,2009-07-06
8267,60162,R99,T2,ericsson,1999-01-01,1999-12-31,1.0,ericsson,peter.bleckert@ericsson.com,22.945,2009-07-23,2010-02-09


### 新增下載狀況：該WI是否有連結、以及是否有下載成功

In [246]:
is_hyperlink = []
is_download = []
for i in wi_overview['Unique_ID']:
    if i in wi_id_without_links:
        is_hyperlink += 'N'
        is_download += 'N'
    else:
        is_hyperlink += 'Y'
        if i in set(result['unique_ID']):
            is_download += 'Y'
        else:
            is_download += 'N'

In [247]:
wi_overview.insert(6, 'is_download', is_download)
wi_overview.insert(6, 'is_hyperlink', is_hyperlink)

### 新增專案完成度：以專案是否完成取代專案完成比率

In [248]:
is_complete = []

for i in wi_overview['%Complete']:
    if i != 1 :
        is_complete += 'N'
    else:
        is_complete += 'Y'

In [249]:
wi_overview.insert(6, 'is_complete', is_complete)
wi_overview = wi_overview.drop('%Complete', axis=1)

### 新增TSG分類：分成RAN、SA、CT

In [250]:
RAN = []
SA = []
CT = []

for i in wi_overview['ResourceNames']:
    
    if 'R' not in str(i) :
        RAN += 'N'
    else:
        RAN += 'Y'
        
    if 'S' not in str(i) :
        SA += 'N'
    else:
        SA += 'Y'    
        
    if 'C' not in str(i) :
        CT += 'N'
    else:
        CT += 'Y'    

In [251]:
wi_overview.insert(2, 'TSG-RAN', RAN)
wi_overview.insert(3, 'TSG-SA', SA)
wi_overview.insert(4, 'TSG-CT', CT)
wi_overview = wi_overview.drop('ResourceNames', axis=1)

In [253]:
wi_overview.to_excel('wi_overview.xlsx', encoding='utf_8_sig', index=False)

C:\Users\Penny\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
